In [14]:
import numpy as np
import pandas as pd

#color_est = pd.read_csv("../data/color2emotion_rescaling.csv",index_col = False)
emotion_est = pd.read_csv("../result/emotion_estimation.csv")
action_est = pd.read_csv("../result/action_estimation.csv")

In [15]:
#color_est

In [16]:
emotion_est


,감정,에너지,회복,순환,정화
0,행복,0.339845,0.264405,0.228558,0.332344
1,우울,0.346771,0.472084,0.350055,0.307193
2,슬픔,0.426716,0.216250,0.303326,0.318329
3,열정,0.345846,0.406059,0.396388,0.401650
4,분노,0.413446,0.457544,0.475200,0.482633
5,두려움,0.333882,0.440799,0.474286,0.441280


In [17]:
action_est

,감정,에너지,회복,순환,정화
0,명상,0.485562,0.409450,0.427934,0.390121
1,산책,0.377276,0.625222,0.501750,0.755757
2,등산,0.411553,0.389895,0.518401,0.402741
3,샤워,0.268861,0.254890,0.377386,0.335477
4,수영,0.460684,0.667701,0.478729,0.313134
5,조깅,0.330085,0.381864,0.475510,0.289971
6,자전거 타기,0.411553,0.270111,0.461505,0.188379
7,요가,0.331387,0.365959,0.492901,0.421356
8,스트레칭,0.325151,0.448834,0.521865,0.518657
9,여행,0.392194,0.316684,0.276691,0.377315


In [18]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
from scipy.stats import pearsonr


def pearson_similarity(a, b):
  return np.dot((a - np.mean(a)), (b - np.mean(b))) / ((np.linalg.norm(a - np.mean(a))) * (np.linalg.norm(b - np.mean(b))))

In [19]:
# 감정 -> 효능 <- 행동 시나리오
class emotion2item():
    def __init__(self ,action_df, emotion_df):
        self.action_df = action_df
        self.emothon_df = emotion_df
             
        
    def sim_item_MAE(self, em_idx):
        em_name , * em_est = self.emothon_df.iloc[em_idx].tolist()
        em_est = np.array(em_est)
        sim_dict = {}
        for idx in self.action_df.index:
            act_name , *act_est = self.action_df.iloc[idx]
            act_est = np.array(act_est)
            sim = np.mean(np.abs(em_est - act_est))
            
            sim_dict[act_name] = sim
            
            print(f'{em_name}과 {act_name}의 MAE : {sim}')
        
        sorted_dict = sorted(sim_dict.items(), key= lambda x:x[1])
        print("-"*20)
        print('{}에서 고양(극복)하는 item은 {}'.format(em_name,sorted_dict[0][0]))    

    def sim_item_Cosine(self, em_idx):
        em_name , * em_est = self.emothon_df.iloc[em_idx].tolist()
        em_est = np.array(em_est)
        sim_dict = {}
        for idx in self.action_df.index:
            act_name , *act_est = self.action_df.iloc[idx]
            act_est = np.array(act_est)
            sim = dot(em_est, act_est)/(norm(em_est)*norm(act_est))
            
            sim_dict[act_name] = sim
            print(f'{em_name}과 {act_name}의 Cosine 유사도 : {sim}')
        
        sorted_dict = sorted(sim_dict.items(),reverse=True, key= lambda x:x[1])
        print("-"*20)
        print('{}에서 고양(극복)하는 item은 {}'.format(em_name,sorted_dict[0][0]))    

    def sim_item_Pearson(self, em_idx):
        em_name , * em_est = self.emothon_df.iloc[em_idx].tolist()
        em_est = np.array(em_est)
        sim_dict = {}
        for idx in self.action_df.index:
            act_name , *act_est = self.action_df.iloc[idx]
            act_est = np.array(act_est)
            sim = pearson_similarity(em_est, act_est)
            
            sim_dict[act_name] = sim
            print(f'{em_name}과 {act_name}의 피어슨 유사도 : {sim}')
        
        sorted_dict = sorted(sim_dict.items(), reverse=True ,key= lambda x:x[1])
        print("-"*20)
        print('{}에서 고양(극복)하는 item은 {}'.format(em_name,sorted_dict[0][0]))    

    def sim_item_callback(self, em_idx,callback,reverse):
        em_name , * em_est = self.emothon_df.iloc[em_idx].tolist()
        em_est = np.array(em_est)
        sim_dict = {}
        for idx in self.action_df.index:
            act_name , *act_est = self.action_df.iloc[idx]
            act_est = np.array(act_est)
            sim = callback(em_est, act_est)
            
            sim_dict[act_name] = sim
            print(f'{em_name}과 {act_name}의 callback 유사도 : {sim}')
        
        sorted_dict = sorted(sim_dict.items(), reverse=reverse ,key= lambda x:x[1])
        print("-"*20)
        print('{}에서 고양(극복)하는 item은 {}'.format(em_name,sorted_dict[0][0]))    


    def max_item_MAE(self, em_idx, n):
        em_name , * em_est = self.emothon_df.iloc[em_idx].tolist()
        em_est = np.array(em_est)
        effs = ['에너지','회복','순환','정화']
        em_dict = {}
        i = 0
        for eff in effs:
            em_dict[eff] = em_est[i]
            i = i + 1
        max_dict = sorted(em_dict.items(), reverse=True,key= lambda x:x[1])[:n]

        sort_em_list = []
        sort_em_est = []
        for x, y in max_dict:
            sort_em_list.append(x)
            sort_em_est.append(y)

        sort_em_df = self.emothon_df[sort_em_list] 
        em_est = np.array(sort_em_df.iloc[em_idx])
        sort_act_df = self.action_df[sort_em_list]
        print(f'{em_name}의 MAX 차원 {sort_em_list}')


        sim_dict = {}
        for idx in sort_act_df.index:
            act_name , *_ = self.action_df.iloc[idx]
            act_est = np.array(sort_act_df.iloc[idx])
            sim = np.mean(np.abs(em_est - act_est))
            sim_dict[act_name] = sim
            print(f'{em_name}과 {act_name}의 MAE : {sim}')
        sorted_dict = sorted(sim_dict.items(), key= lambda x:x[1])
        print("-"*20)
        print('{}에서 고양(극복)하는 item은 {}'.format(em_name,sorted_dict[0][0]))    

            

    def complete_item(self, em_idx, n):
        pass

In [20]:
# 감정과 가장 유사한 item을 추천 using MAE
XYZ = emotion2item(action_est, emotion_est)
for i in range(0,5):
    XYZ.sim_item_MAE(i)
    print('\n')


행복과 명상의 MAE : 0.13697848282754424
행복과 산책의 MAE : 0.2737134192138911
행복과 등산의 MAE : 0.13935956172645098
행복과 샤워의 MAE : 0.05811514519155026
행복과 수영의 MAE : 0.19837871380150318
행복과 조깅의 MAE : 0.10413576103746894
행복과 자전거 타기의 MAE : 0.1135811172425747
행복과 요가의 MAE : 0.11584193073213107
행복과 스트레칭의 MAE : 0.16968589089810854
행복과 여행의 MAE : 0.049433032050728805
--------------------
행복에서 가장 유사한 item은 여행


우울과 명상의 MAE : 0.09055782482028009
우울과 산책의 MAE : 0.19597539305686956
우울과 등산의 MAE : 0.10271600261330607
우울과 샤워의 MAE : 0.08767961710691449
우울과 수영의 MAE : 0.11103615164756775
우울과 조깅의 MAE : 0.062395706772804246
우울과 자전거 타기의 MAE : 0.12425478361546993
우울과 요가의 MAE : 0.09462959691882133
우울과 스트레칭의 MAE : 0.10703612491488457
우울과 여행의 MAE : 0.08607696741819382
--------------------
우울에서 가장 유사한 item은 조깅


슬픔과 명상의 MAE : 0.11211127787828443
슬픔과 산책의 MAE : 0.27356601879000664
슬픔과 등산의 MAE : 0.12207364290952681
슬픔과 샤워의 MAE : 0.07192570716142653
슬픔과 수영의 MAE : 0.16650443151593208
슬픔과 조깅의 MAE : 0.11569666489958763
슬픔과 자전거 타기의 MAE 

In [21]:
# 감정과 가장 유사한 item을 추천 using 코사인 유사도
XYZ = emotion2item(action_est, emotion_est)
for i in range(0,5):
    XYZ.sim_item_Cosine(i)
    print('\n')


행복과 명상의 Cosine 유사도 : 0.9874503983326837
행복과 산책의 Cosine 유사도 : 0.9596853252752822
행복과 등산의 Cosine 유사도 : 0.9678381915581427
행복과 샤워의 Cosine 유사도 : 0.9648541570911945
행복과 수영의 Cosine 유사도 : 0.9333203167583458
행복과 조깅의 Cosine 유사도 : 0.9430144945877919
행복과 자전거 타기의 Cosine 유사도 : 0.9178200019029611
행복과 요가의 Cosine 유사도 : 0.9609486223038732
행복과 스트레칭의 Cosine 유사도 : 0.958317447429442
행복과 여행의 Cosine 유사도 : 0.9996885185267267
--------------------
행복에서 가장 유사한 item은 여행


우울과 명상의 Cosine 유사도 : 0.981520225613771
우울과 산책의 Cosine 유사도 : 0.9567511852682071
우울과 등산의 Cosine 유사도 : 0.9736140435768021
우울과 샤워의 Cosine 유사도 : 0.9574443936604594
우울과 수영의 Cosine 유사도 : 0.9945038199834583
우울과 조깅의 Cosine 유사도 : 0.9782727506032577
우울과 자전거 타기의 Cosine 유사도 : 0.9339599854937265
우울과 요가의 Cosine 유사도 : 0.9668033640675574
우울과 스트레칭의 Cosine 유사도 : 0.9670838166704424
우울과 여행의 Cosine 유사도 : 0.9680948872163788
--------------------
우울에서 가장 유사한 item은 수영


슬픔과 명상의 Cosine 유사도 : 0.984350199822908
슬픔과 산책의 Cosine 유사도 : 0.910518238232133
슬픔과 등산의 Cosine 유사도 : 0.9

In [22]:
# 감정과 가장 유사한 item을 추천 using 피어슨 유사도
XYZ = emotion2item(action_est, emotion_est)
for i in range(0,5):
    XYZ.sim_item_Pearson(i)
    print('\n')


행복과 명상의 피어슨 유사도 : 0.26181551742446674
행복과 산책의 피어슨 유사도 : 0.040554072877389255
행복과 등산의 피어슨 유사도 : -0.6780125398830655
행복과 샤워의 피어슨 유사도 : -0.39802529172156054
행복과 수영의 피어슨 유사도 : -0.5428403768559749
행복과 조깅의 피어슨 유사도 : -0.9392092727383099
행복과 자전거 타기의 피어슨 유사도 : -0.4175529403217898
행복과 요가의 피어슨 유사도 : -0.64662378355818
행복과 스트레칭의 피어슨 유사도 : -0.5191703276596763
행복과 여행의 피어슨 유사도 : 0.9978026451721016
--------------------
행복에서 가장 유사한 item은 여행


우울과 명상의 피어슨 유사도 : -0.09623019388591199
우울과 산책의 피어슨 유사도 : -0.006055962691407941
우울과 등산의 피어슨 유사도 : -0.29297148334664075
우울과 샤워의 피어슨 유사도 : -0.6184462753195303
우울과 수영의 피어슨 유사도 : 0.9665845486880715
우울과 조깅의 피어슨 유사도 : 0.2945701777930007
우울과 자전거 타기의 피어슨 유사도 : -0.06395282179226373
우울과 요가의 피어슨 유사도 : -0.33519051426264307
우울과 스트레칭의 피어슨 유사도 : -0.14961726293758054
우울과 여행의 피어슨 유사도 : -0.40563168223655227
--------------------
우울에서 가장 유사한 item은 수영


슬픔과 명상의 피어슨 유사도 : 0.7622267591194023
슬픔과 산책의 피어슨 유사도 : -0.6066274471699394
슬픔과 등산의 피어슨 유사도 : 0.05047036516358566
슬픔과 샤워의 피어슨 유사도 : 0.00

In [23]:
# 추청치가 가장 높은 차원 1개에 대해서 MAE를 이용해서 추천
XYZ = emotion2item(action_est, emotion_est)
for idx in range(0,5):
    XYZ.max_item_MAE(idx,1)
    print("\n\n")

행복의 MAX 차원 ['에너지']
행복과 명상의 MAE : 0.14571650326251984
행복과 산책의 MAE : 0.037430763244628906
행복과 등산의 MAE : 0.07170780003070831
행복과 샤워의 MAE : 0.07098431140184408
행복과 수영의 MAE : 0.12083855271339411
행복과 조깅의 MAE : 0.009759873151779175
행복과 자전거 타기의 MAE : 0.07170780003070831
행복과 요가의 MAE : 0.008458182215690668
행복과 스트레칭의 MAE : 0.014694646000862177
행복과 여행의 MAE : 0.052348554134368896
--------------------
행복에서 가장 유사한 item은 요가



우울의 MAX 차원 ['회복']
우울과 명상의 MAE : 0.06263473629951477
우울과 산책의 MAE : 0.15313816070556646
우울과 등산의 MAE : 0.0821889340877533
우울과 샤워의 MAE : 0.2171939834952354
우울과 수영의 MAE : 0.19561676681041723
우울과 조깅의 MAE : 0.0902202427387237
우울과 자전거 타기의 MAE : 0.20197353512048716
우울과 요가의 MAE : 0.10612553358077997
우울과 스트레칭의 MAE : 0.023250669240951483
우울과 여행의 MAE : 0.15540018677711487
--------------------
우울에서 가장 유사한 item은 스트레칭



슬픔의 MAX 차원 ['에너지']
슬픔과 명상의 MAE : 0.0588461309671402
슬픔과 산책의 MAE : 0.04943960905075073
슬픔과 등산의 MAE : 0.015162572264671326
슬픔과 샤워의 MAE : 0.15785468369722372
슬픔과 수영의 MAE : 0.03396

In [24]:
# 추청치가 가장 높은 차원 2개에 대해서 MAE를 이용해서 추천
XYZ = emotion2item(action_est, emotion_est)
for idx in range(0,5):
    XYZ.max_item_MAE(idx,2)
    print("\n\n")

행복의 MAX 차원 ['에너지', '정화']
행복과 명상의 MAE : 0.10174679756164551
행복과 산책의 MAE : 0.23042197525501254
행복과 등산의 MAE : 0.07105260342359546
행복과 샤워의 MAE : 0.037058878690004377
행복과 수영의 MAE : 0.07002381235361096
행복과 조깅의 MAE : 0.026066124439239502
행복과 자전거 타기의 MAE : 0.10783604532480237
행복과 요가의 MAE : 0.048735313117504175
행복과 스트레칭의 MAE : 0.10050388425588613
행복과 여행의 MAE : 0.04865982383489609
--------------------
행복에서 가장 유사한 item은 조깅



우울의 MAX 차원 ['회복', '순환']
우울과 명상의 MAE : 0.07025670260190967
우울과 산책의 MAE : 0.15241666138172155
우울과 등산의 MAE : 0.12526725232601169
우울과 샤워의 MAE : 0.12226235494017598
우울과 수영의 MAE : 0.16214541345834735
우울과 조깅의 MAE : 0.107837438583374
우울과 자전거 타기의 MAE : 0.15671152248978612
우울과 요가의 MAE : 0.12448588013648987
우울과 스트레칭의 MAE : 0.09753036499023438
우울과 여행의 MAE : 0.11438196152448654
--------------------
우울에서 가장 유사한 item은 명상



슬픔의 MAX 차원 ['에너지', '정화']
슬픔과 명상의 MAE : 0.065318688750267
슬픔과 산책의 MAE : 0.24343347549438477
슬픔과 등산의 MAE : 0.049787066876888275
슬픔과 샤워의 MAE : 0.08750114217400551
슬픔과 수영의 

In [25]:
# 추청치가 가장 높은 차원 3개에 대해서 MAE를 이용해서 추천
XYZ = emotion2item(action_est, emotion_est)
for idx in range(0,5):
    XYZ.max_item_MAE(idx,3)
    print("\n\n")

행복의 MAX 차원 ['에너지', '정화', '회복']
행복과 명상의 MAE : 0.11617933462063472
행복과 산책의 MAE : 0.27388708541790646
행복과 등산의 MAE : 0.0891984725991885
행복과 샤워의 MAE : 0.027877531945705414
행복과 수영의 MAE : 0.18111451218525568
행복과 조깅의 MAE : 0.056530383725961066
행복과 자전거 타기의 MAE : 0.07379256685574849
행복과 요가의 MAE : 0.06634141256411875
행복과 스트레칭의 MAE : 0.1284787481029829
행복과 여행의 MAE : 0.049866201976935066
--------------------
행복에서 가장 유사한 item은 샤워



우울의 MAX 차원 ['회복', '순환', '에너지']
우울과 명상의 MAE : 0.09310135245323185
우울과 산책의 MAE : 0.11177941163380946
우울과 등산의 MAE : 0.10510548452536268
우울과 샤워의 MAE : 0.1074782907962799
우울과 수영의 MAE : 0.1460678428411484
우울과 조깅의 MAE : 0.07745353380839026
우울과 자전거 타기의 MAE : 0.1260683313012123
우울과 요가의 MAE : 0.08811859786510468
우울과 스트레칭의 MAE : 0.07222707569599152
우울과 여행의 MAE : 0.09139554202556611
--------------------
우울에서 가장 유사한 item은 스트레칭



슬픔의 MAX 차원 ['에너지', '정화', '순환']
슬픔과 명상의 MAE : 0.08508184055487313
슬픔과 산책의 MAE : 0.22843052943547568
슬픔과 등산의 MAE : 0.10488305985927582
슬픔과 샤워의 MAE : 0.0830208

In [26]:
# 추청치가 가장 높은 차원 4개에 대해서 MAE를 이용해서 추천
XYZ = emotion2item(action_est, emotion_est)
for idx in range(0,5):
    XYZ.max_item_MAE(idx,1)
    print("\n\n")

행복의 MAX 차원 ['에너지']
행복과 명상의 MAE : 0.14571650326251984
행복과 산책의 MAE : 0.037430763244628906
행복과 등산의 MAE : 0.07170780003070831
행복과 샤워의 MAE : 0.07098431140184408
행복과 수영의 MAE : 0.12083855271339411
행복과 조깅의 MAE : 0.009759873151779175
행복과 자전거 타기의 MAE : 0.07170780003070831
행복과 요가의 MAE : 0.008458182215690668
행복과 스트레칭의 MAE : 0.014694646000862177
행복과 여행의 MAE : 0.052348554134368896
--------------------
행복에서 가장 유사한 item은 요가



우울의 MAX 차원 ['회복']
우울과 명상의 MAE : 0.06263473629951477
우울과 산책의 MAE : 0.15313816070556646
우울과 등산의 MAE : 0.0821889340877533
우울과 샤워의 MAE : 0.2171939834952354
우울과 수영의 MAE : 0.19561676681041723
우울과 조깅의 MAE : 0.0902202427387237
우울과 자전거 타기의 MAE : 0.20197353512048716
우울과 요가의 MAE : 0.10612553358077997
우울과 스트레칭의 MAE : 0.023250669240951483
우울과 여행의 MAE : 0.15540018677711487
--------------------
우울에서 가장 유사한 item은 스트레칭



슬픔의 MAX 차원 ['에너지']
슬픔과 명상의 MAE : 0.0588461309671402
슬픔과 산책의 MAE : 0.04943960905075073
슬픔과 등산의 MAE : 0.015162572264671326
슬픔과 샤워의 MAE : 0.15785468369722372
슬픔과 수영의 MAE : 0.03396